<a href="https://colab.research.google.com/github/desirel-dev/1/blob/main/LLaVa/Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes torch pillow requests


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

In [2]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model_id = "llava-hf/llava-1.5-7b-hf"

In [3]:

processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")



Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

## Prepare image and text for the model

In [16]:
def generate_answer(image: Image.Image, prompt_text: str) -> str:
    """
    Przyjmuje obraz PIL i tekst promptu, generuje odpowiedź modelu LLaVA.

    Args:
        image: Obiekt PIL Image.
        prompt_text: Pytanie użytkownika.

    Returns:
        Wygenerowana odpowiedź tekstowa.
    """
    if image is None:
        return "Błąd: Proszę załadować obraz."

    if not prompt_text:
        return "Błąd: Proszę wpisać pytanie."

    # 1. Konstrukcja pełnego promptu LLaVA
    full_prompt = f"USER: <image>\n{prompt_text}\nASSISTANT:"

    # 2. Tokenizacja i przygotowanie danych wejściowych
    # W LLava obraz jest przekazywany w liście, nawet jeśli jest jeden
    inputs = processor(
        text=full_prompt,
        images=image,
        padding="longest",
        return_tensors="pt"
    )

    # 3. Przeniesienie tokenów wejściowych na urządzenie, na którym jest model
    # To jest kluczowe dla poprawnej pracy z device_map="auto"
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 4. Generowanie odpowiedzi
    # Maksymalna liczba nowych tokenów zwiększona dla lepszych wyników
    output = model.generate(**inputs, max_new_tokens=100)

    # 5. Dekodowanie i czyszczenie wyników
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

    # Wyodrębnienie tylko odpowiedzi Asystenta
    response = generated_text.split("ASSISTANT:")[-1].strip()

    return response

print("Główna funkcja generująca zdefiniowana.")

Główna funkcja generująca zdefiniowana.


In [ ]:
# Przykład domyślnego obrazu dla interfejsu
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
default_image = Image.open(requests.get(url, stream=True).raw)

# Definicja interfejsu Gradio
iface = gr.Interface(
    fn=generate_answer, # Nasza funkcja, która wykonuje całą logikę
    inputs=[
        gr.Image(type="pil", label="Załaduj Obraz", value=default_image),
        gr.Textbox(lines=2, label="Twoje Pytanie (Prompt)", value="Please describe this image.")
    ],
    outputs=gr.Textbox(label="Odpowiedź LLaVA"),
    title="LLaVA 1.5-7B Chat - Kwantyzacja 4-bitowa (HF/Gradio)",
    description="Załaduj obraz i zadaj pytanie. Model LLaVA wygeneruje odpowiedź."
)

# Uruchomienie interfejsu
# Gradio automatycznie uruchomi się w przeglądarce lub wygeneruje link do tunnelingu w Colab
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0eccacbe1037ea8a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
